In [1]:
import os
import pandas as pd

RATE_SPEECHES_FILE = "joined.parquet.gzip"
assert os.path.exists(RATE_SPEECHES_FILE), f"file not present: {RATE_SPEECHES_FILE}"

rate_speeches = pd.read_parquet(RATE_SPEECHES_FILE)


In [2]:
rate_speeches.head()

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,speech_id,when_speech,who,what_title,what_frequencies,what_language,what_weblink,what_type,Meeting Dates,Rate Change Date,Main refinancing operations - fixed rate tenders,Change (%),Direction,is_col2_earlier,same_date,extracted_text
0,2748,1999-03-04,Willem F. Duisenberg,Introductory statement with Q&A,"{""euro"":45,""question"":33,""rate"":30,""duisenberg...",EN,https://www.ecb.europa.eu/press/pressconf/1999...,P,1999-03-04,None,3.0,0.0,No Change,False,True,"Willem F. Duisenberg, President of the Europea..."
1,80,1999-03-08,Christian Noyer,First experiences with the euro,"{""monetary"":64,""euro"":51,""price"":48,""policy"":4...",EN,https://www.ecb.europa.eu/press/key/date/1999/...,S,1999-04-08,4/9/99,2.5,-0.5,Decrease,False,False,"Speech by Christian Noyer, Vice-President of t..."
2,81,1999-03-09,Tommaso Padoa-Schioppa,The euro: significance for Europe and beyond,"{""euro"":29,""currency"":20,""monetary"":15,""single...",EN,https://www.ecb.europa.eu/press/key/date/1999/...,S,1999-04-08,4/9/99,2.5,-0.5,Decrease,False,False,Speech by Tommaso Padoa-Schioppa Member of the...
3,82,1999-03-10,Eugenio Domingo Solans,The euro as a new world currency,"{""euro"":63,""currency"":29,""area"":23,""monetary"":...",EN,https://www.ecb.europa.eu/press/key/date/1999/...,S,1999-04-08,4/9/99,2.5,-0.5,Decrease,False,False,"Eugenio Domingo Solans, Member of the Executiv..."
4,83,1999-03-12,Willem F. Duisenberg,The Eurosystem's strategy for the euro,"{""monetary"":50,""stability"":35,""policy"":35,""pri...",EN,https://www.ecb.europa.eu/press/key/date/1999/...,S,1999-04-08,4/9/99,2.5,-0.5,Decrease,False,False,"Speech by Dr. Willem F. Duisenberg, President ..."


In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# analyze text lengths

from nltk import word_tokenize

def rate_speeches_text_length():
  rate_speeches['text_length'] = rate_speeches['extracted_text'].apply(lambda x: len(word_tokenize(x)))

if os.path.exists('rate_speeches.text_length.parquet.gzip'):
  rate_speeches = pd.read_parquet('rate_speeches.text_length.parquet.gzip')
else:
  rate_speeches_text_length()
  rate_speeches.to_parquet('rate_speeches.text_length.parquet.gzip', compression='gzip')

print(rate_speeches['text_length'].describe())

count     3434.000000
mean      3307.266453
std       2296.389089
min          0.000000
25%       1809.000000
50%       2753.000000
75%       4115.750000
max      26091.000000
Name: text_length, dtype: float64


In [5]:
percentile_95 = rate_speeches['text_length'].quantile(0.95)
print(f"95th Percentile of text_length: {percentile_95}")

95th Percentile of text_length: 7619.0


In [7]:
import nltk
from nltk.tokenize import sent_tokenize

def rate_speeches_sent_tokenize():
  if os.path.exists('rate_speeches.sent_tokenize.parquet.gzip'):
    return pd.read_parquet('rate_speeches.sent_tokenize.parquet.gzip')
  else:
    rate_speeches_sent_tokenize = rate_speeches.copy()
    rate_speeches_sent_tokenize['extracted_text'] = rate_speeches_sent_tokenize['extracted_text'].apply(sent_tokenize)
    rate_speeches_sent_tokenize.to_parquet('rate_speeches.sent_tokenize.parquet.gzip', compression='gzip')
    return rate_speeches_sent_tokenize

tokenized_rate_speeches = rate_speeches_sent_tokenize()
tokenized_rate_speeches['extracted_text'].head()


,extracted_text
0,"[Willem F. Duisenberg, President of the Europe..."
1,"[Speech by Christian Noyer, Vice-President of ..."
2,[Speech by Tommaso Padoa-Schioppa Member of th...
3,"[Eugenio Domingo Solans, Member of the Executi..."
4,"[Speech by Dr. Willem F. Duisenberg, President..."


In [8]:
tokenized_rate_speeches[tokenized_rate_speeches["extracted_text"].apply(lambda x: len(x) == 0)]

,speech_id,when_speech,who,what_title,what_frequencies,what_language,what_weblink,what_type,Meeting Dates,Rate Change Date,Main refinancing operations - fixed rate tenders,Change (%),Direction,is_col2_earlier,same_date,extracted_text,text_length
1171,1208,2009-12-11,Gertrude Tumpel-Gugerell,"The Way Forward with Monetary, Fiscal and Mac...","{""financial"":18,""policy"":15,""fiscal"":14,""crisi...",EN,https://www.ecb.europa.eu/press/key/date/2009/...,S,2010-01-14,None,1.0,0.0,No Change,False,False,[],0


In [10]:
import os
import nltk
import gensim
import numpy as np
import unicodedata

from itertools import groupby
from unicodedata import category as unicat

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from nltk.chunk import tree2conlltags
from nltk.probability import FreqDist
from nltk.chunk.regexp import RegexpParser
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.utils import pad_sequences


class KeyphraseExtractor(BaseEstimator, TransformerMixin):
    """
    Extract adverbial and adjective phrases, and transform
    documents into lists of these keyphrases, with a total
    keyphrase lexicon limited by the nfeatures parameter
    and a document length limited/padded to doclen
    """
    def __init__(self, nfeatures=100000, doclen=7619):
        self.grammar = r'KT: {(<RB.> <JJ.*>|<VB.*>|<RB.*>)|(<JJ> <NN.*>)}'
        # self.grammar = r'KT: {(<RB.*> <VB.>|<RB.>|<JJ.> <NN.*>)}'
        # self.grammar = r'KT: {<RB.>|<JJ.>}'
        self.chunker = RegexpParser(self.grammar)
        self.nfeatures = nfeatures
        self.doclen = doclen
        self._curr = 0

    def normalize(self, sent):
        """
        Removes punctuation from a tokenized/tagged sentence and
        lowercases words.
        """
        is_punct = lambda word: all(unicat(c).startswith('P') for c in word)
        sent = filter(lambda t: not is_punct(t[0]), sent)
        sent = map(lambda t: (t[0].lower(), t[1]), sent)
        return list(sent)

    def extract_candidate_phrases(self, sents):
        """
        For a document, parse sentences using our chunker created by
        our grammar, converting the parse tree into a tagged sequence.
        Extract phrases, rejoin with a space, and yield the document
        represented as a list of it's keyphrases.
        """

        self._curr += 1

        for i, sent in enumerate(sents):
            tokens = word_tokenize(sent)
            pos_tags = pos_tag(tokens)
            normalized = self.normalize(pos_tags)
            print(f"[{self._curr}] sent #{i}: {sent}")
            print(f"[{self._curr}] \t {normalized}")

            chunks = tree2conlltags(self.chunker.parse(normalized))
            if not chunks or all(chunk[-1] == 'O' for chunk in chunks):
                #print(f"No valid chunks found in sentence: {sent}")
                continue

            phrases = [
                " ".join(word for word, pos, chunk in group).lower()
                for key, group in groupby(
                    chunks, lambda term: term[-1] != 'O'
                ) if key
            ]
            for phrase in phrases:
                yield phrase

    def fit(self, documents, y=None):
        return self

    def get_lexicon(self, keydocs):
        """
        Build a lexicon of size nfeatures
        """
        keyphrases = [keyphrase for doc in keydocs for keyphrase in doc]
        print("Keyphrases:", keyphrases[:5])
        fdist = FreqDist(keyphrases)
        counts = fdist.most_common(self.nfeatures)
        print("Frequency counts:", counts[:5])
        lexicon = [phrase for phrase, count in counts]
        return {phrase: idx+1 for idx, phrase in enumerate(lexicon)}

    def clip(self, keydoc, lexicon):
        """
        Remove keyphrases from documents that aren't in the lexicon
        """
        return [lexicon[keyphrase] for keyphrase in keydoc
            if keyphrase in lexicon.keys()]

    def transform(self, documents):
        docs = [list(self.extract_candidate_phrases(doc)) for doc in documents]
        lexicon = self.get_lexicon(docs)
        clipped = [list(self.clip(doc, lexicon)) for doc in docs]
        return pad_sequences(clipped, maxlen=self.doclen)

In [12]:
import os
import time
import numpy as np

from functools import wraps

from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding

N_FEATURES = 10000
DOC_LEN = 7619
NUM_CLASSES = 3

def build_lstm():
    lstm = Sequential()
    lstm.add(Embedding(input_dim=N_FEATURES + 1, output_dim=128))
    lstm.add(Dropout(0.4))
    lstm.add(LSTM(units=200, recurrent_dropout=0.2, dropout=0.2))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(NUM_CLASSES, activation='softmax')) # softmax for mult-class classification
    lstm.compile(
        loss='categorical_crossentropy', # target vals are 0, 1 or 2
        optimizer='adam',
        metrics=['accuracy']
    )
    return lstm

In [13]:
import joblib
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_val_score
from tensorflow.keras.utils import to_categorical

# define the target label mapping
label_mapping = {"No Change": 0, "Increase": 1, "Decrease": 2}

def documents(df):
    texts = df['extracted_text'].values
    return texts

def binarize(df):
    labels = to_categorical(df['Direction'].map(label_mapping).values, num_classes=NUM_CLASSES)
    return labels

def train_model(df, model, saveto=None, cv=12, **kwargs):
    """
    Trains model from corpus at specified path; fitting the model on the full data and writing it to disk at the saveto directory if specified.
    Returns the scores.
    """
    # Load the corpus data and labels for classification
    X = documents(df)
    print("Shape after tokenization:", len(X))

    y = binarize(df)

    # Compute cross validation scores
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

    # Fit the model on entire data set
    model.fit(X, y)

    # Write to disk if specified
    if saveto:
        # have to save the keras part using keras' save method
        model.steps[-1][1].model.save(saveto['keras_model'])
        model.steps.pop(-1)
        # ... and use joblib to save the rest of the pipeline
        joblib.dump(model, saveto['sklearn_pipe'])

    # Return scores as well as training time via decorator
    return scores

In [14]:
from sklearn.model_selection import train_test_split

X = tokenized_rate_speeches['extracted_text']
y = tokenized_rate_speeches['Direction']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
class DebugTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print("Data Shape Before LSTM:", X.shape)
        print("Sample Data Before LSTM:", X[0])  # Print a sample
        return X

In [16]:
!pip install scikeras

In [17]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from scikeras.wrappers import KerasClassifier

pipeline = Pipeline([
    ('keyphrases', KeyphraseExtractor(nfeatures=N_FEATURES, doclen=DOC_LEN)),
    ('debug', DebugTransformer()),
    ('lstm', KerasClassifier(model=build_lstm, epochs=4, batch_size=128))
])

scores, delta = train_model(tokenized_rate_speeches, pipeline, cv=4)

print('Mean score: {}'.format(np.mean(scores)))
print('Total fit time: {:0.2f} seconds'.format(delta))

Streaming output truncated to the last 5000 lines.
[2543] sent #4: Undoubtedly the euro is the most important element of European integration.
[2543] 	 [('undoubtedly', 'RB'), ('the', 'DT'), ('euro', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('most', 'RBS'), ('important', 'JJ'), ('element', 'NN'), ('of', 'IN'), ('european', 'JJ'), ('integration', 'NN')]
[2543] sent #5: It's not only economics or financial issues; it's that this is the real instrument in terms of cooperation.
[2543] 	 [('it', 'PRP'), ("'s", 'VBZ'), ('not', 'RB'), ('only', 'RB'), ('economics', 'NNS'), ('or', 'CC'), ('financial', 'JJ'), ('issues', 'NNS'), ('it', 'PRP'), ("'s", 'VBZ'), ('that', 'IN'), ('this', 'DT'), ('is', 'VBZ'), ('the', 'DT'), ('real', 'JJ'), ('instrument', 'NN'), ('in', 'IN'), ('terms', 'NNS'), ('of', 'IN'), ('cooperation', 'NN')]
[2543] sent #6: Despite the difficulties, because undoubtedly we had the debt crisis, as you have said, today the euro has delivered financial and economic stability to the euro z

ValueError: 
All the 4 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.10/dist-